# Lesson 3 Exercise 3: Focus on Clustering Columns
![](../images/cassandralogo.png)

Walk through the basics of creating a table with a good Primary Key and Clustering Columns in Apache Cassandra, inserting rows of data, and doing a simple CQL query to validate the information. Remember, replace ##### with your own code.

In [ ]:
from src.database import (
    get_cs_cluster,
    get_cs_session,
    close_cs_session,
    shutdown_cs_cluster,
    insert_cs_rows,
    create_cs_keyspace,
    create_cs_table,
    drop_cs_table,
    drop_cs_keyspace,
    set_cs_keyspace,
)

cluster = get_cs_cluster()
session = get_cs_session(cluster)
keyspace_name = "my_keyspace"
drop_cs_keyspace(session, keyspace_name)  # drop any existing keyspace (nice for reruns)
create_cs_keyspace(session, keyspace_name)
set_cs_keyspace(session, keyspace_name)  # set the keyspace to the one we just created
tables = set()

Imagine we would like to start creating a new Music Library of albums. 

We want to ask 1 question of our data:
1. Give me all the information from the music library about a given album
```sql
SELECT * FROM music_library WHERE album_name="Close To You"
```

### Here is the data:
![](images/table4.png)

How should we model this data? What should be our Primary Key and Partition Key? 

In [ ]:
table_name_alb = "album_library"
tables.add(table_name_alb)
drop_cs_table(session, table_name_alb)  # drop the table if it exists
column_names = [
    "year",
    "artist_name",
    "album_name",
    "city",
]
column_types = [
    "int",
    "text",
    "text",
    "text",
]
columns_alb = dict(zip(column_names, column_types))
primary_keys = ["album_name", "artist_name", "year"]
create_cs_table(
    session,
    table_name_alb,
    columns_alb,
    primary_keys,
)

### Insert data into the table

In [ ]:
insert_cs_rows(
    session,
    table_name_alb,
    list(columns_alb.keys()),
    [
        (1970, "The Beatles", "Let It Be", "Liverpool"),
        (1965, "The Beatles", "Rubber Soul", "Oxford"),
        (1965, "The Who", "My Generation", "London"),
        (1966, "The Monkees", "The Monkees", "Los Angeles"),
        (1970, "The Carpenters", "Close To You", "San Diego"),
        (1964, "The Beatles", "Beatles For Sale", "London"),
    ],
)

### Validate the Data Model -- Did it work? 
`select * from album_library WHERE album_name="Close To You"`

In [ ]:
query = "SELECT * FROM album_library WHERE album_name='Close To You'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist_name, row.album_name, row.city, row.year)

### Your output should be:
('The Carpenters', 'Close to You', 'San Diego', 1970)

### OR
('The Carpenters', 'Close to You', 1970, 'San Diego') 

### Drop the table

In [ ]:
for table in tables:
    drop_cs_table(session, table)
close_cs_session(session)
shutdown_cs_cluster(cluster)